# How to update azure vm public IP address

In [4]:
# Import the needed credential and management objects from the libraries.
import os
import datetime
from azure.identity import AzureCliCredential, InteractiveBrowserCredential, ClientSecretCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute import ComputeManagementClient


In [5]:
# Set up
'''
TENANT_ID = "794d74bb-9ae1-4541-9e91-7c6b68abe8cd"
SUBSCRIPTION_ID = "ffc6f428-bef1-4059-ad8f-d772730c3681"
RESOURCE_GROUP = "mugen"
LOCATION = "japaneast"
'''
#select Microsoft Entra ID. If you don't see Microsoft Entra ID here, use the search box to find it.
#Find the Tenant ID in the Basic information section of the Overview screen.
TENANT_ID = "17b4ebf0-7d86-4a9b-904a-d384d6f1246f"
SUBSCRIPTION_ID = "d40c7e06-9290-4055-8fa6-0ab11d40ff93"
RESOURCE_GROUP = "kexin"
LOCATION = "japaneast"

# credential 1: interactive browser credential
credential = InteractiveBrowserCredential(tenant_id=TENANT_ID)

# credential 2: client secret credential
# CLIENT_ID = "c06d21e1-39a0-4956-878d-d54d9e1ba57d"
# CLIENT_ST = "<Client_ST>"  # client secret should not be checked in for security reason

# credential = ClientSecretCredential(
#     client_id=CLIENT_ID,
#     client_secret=CLIENT_ST,
#     tenant_id=TENANT_ID,
# )

# get clients to be used
resource_client = ResourceManagementClient(credential, SUBSCRIPTION_ID)
compute_client = ComputeManagementClient(credential, SUBSCRIPTION_ID)
network_client = NetworkManagementClient(credential, SUBSCRIPTION_ID)

In [6]:
resources = list(resource_client.resources.list())
for resource in resources:
    print(resource.as_dict())

{'id': '/subscriptions/d40c7e06-9290-4055-8fa6-0ab11d40ff93/resourceGroups/KEXIN/providers/Microsoft.Compute/disks/kexin_disk1_4c08d1cf0712497cb9c5fd30f42e351e', 'name': 'kexin_disk1_4c08d1cf0712497cb9c5fd30f42e351e', 'type': 'Microsoft.Compute/disks', 'location': 'japaneast', 'managed_by': '/subscriptions/d40c7e06-9290-4055-8fa6-0ab11d40ff93/resourceGroups/kexin/providers/Microsoft.Compute/virtualMachines/kexin', 'sku': {'name': 'Premium_LRS', 'tier': 'Premium'}}
{'id': '/subscriptions/d40c7e06-9290-4055-8fa6-0ab11d40ff93/resourceGroups/kexin/providers/Microsoft.Network/networkSecurityGroups/kexin-nsg', 'name': 'kexin-nsg', 'type': 'Microsoft.Network/networkSecurityGroups', 'location': 'japaneast'}
{'id': '/subscriptions/d40c7e06-9290-4055-8fa6-0ab11d40ff93/resourceGroups/NetworkWatcherRG/providers/Microsoft.Network/networkWatchers/NetworkWatcher_japaneast', 'name': 'NetworkWatcher_japaneast', 'type': 'Microsoft.Network/networkWatchers', 'location': 'japaneast'}
{'id': '/subscriptions

## Create a new public IP address

In [7]:
new_ip_name = datetime.datetime.now().strftime("%Y_%m%d_%H%M")
new_ip = network_client.public_ip_addresses.begin_create_or_update(
    resource_group_name=RESOURCE_GROUP, 
    public_ip_address_name=new_ip_name, 
    parameters={
        'location':LOCATION,
        'sku':{'name': 'standard'},
        'public_ip_allocation_method':'static',
        'public_ip_address_version':'ipv4',
    }
)
new_ip = new_ip.result()
new_ip_address = new_ip.ip_address
print(new_ip.as_dict())
print(f"New IP address is: {new_ip_address}")

{'id': '/subscriptions/d40c7e06-9290-4055-8fa6-0ab11d40ff93/resourceGroups/kexin/providers/Microsoft.Network/publicIPAddresses/2024_0718_0018', 'name': '2024_0718_0018', 'type': 'Microsoft.Network/publicIPAddresses', 'location': 'japaneast', 'sku': {'name': 'Standard', 'tier': 'Regional'}, 'etag': 'W/"47f26466-b237-4e25-b644-c424e4477d10"', 'public_ip_allocation_method': 'Static', 'public_ip_address_version': 'IPv4', 'ddos_settings': {'protection_mode': 'VirtualNetworkInherited'}, 'ip_tags': [], 'ip_address': '48.218.144.2', 'idle_timeout_in_minutes': 4, 'resource_guid': '34c1da91-0641-4ae6-bd63-9c7123ada8df', 'provisioning_state': 'Succeeded'}
New IP address is: 48.218.144.2


## Get IP configration and old IP name

In [8]:
# get network interface
vms = compute_client.virtual_machines.list_all()
vm_mugen = list(vms)[0]
ni_ref = list(vm_mugen.network_profile.network_interfaces)[0]
ni_name = os.path.basename(ni_ref.id)
ni = network_client.network_interfaces.get(RESOURCE_GROUP, ni_name)

# get IP config and old IP name
ip_config = ni.ip_configurations[0]
print(f"IP config dict:\n {ip_config.as_dict()}")
old_ip = ip_config.public_ip_address
old_ip_name = os.path.basename(old_ip.id)
print(f"Old IP name: {old_ip_name}")

# get network security group, should keep current nsg settings when update network interface
nsg = ni.network_security_group
print(f"Network security group dict:\n {nsg.as_dict()}")

IP config dict:
 {'id': '/subscriptions/d40c7e06-9290-4055-8fa6-0ab11d40ff93/resourceGroups/kexin/providers/Microsoft.Network/networkInterfaces/kexin105/ipConfigurations/ipconfig1', 'name': 'ipconfig1', 'etag': 'W/"16b50aef-cb86-44e8-9a83-b27a3643c5da"', 'type': 'Microsoft.Network/networkInterfaces/ipConfigurations', 'private_ip_address': '10.0.0.4', 'private_ip_allocation_method': 'Dynamic', 'private_ip_address_version': 'IPv4', 'subnet': {'id': '/subscriptions/d40c7e06-9290-4055-8fa6-0ab11d40ff93/resourceGroups/kexin/providers/Microsoft.Network/virtualNetworks/kexin-vnet/subnets/default'}, 'primary': True, 'public_ip_address': {'id': '/subscriptions/d40c7e06-9290-4055-8fa6-0ab11d40ff93/resourceGroups/kexin/providers/Microsoft.Network/publicIPAddresses/2024_0120_2216'}, 'provisioning_state': 'Succeeded'}
Old IP name: 2024_0120_2216
Network security group dict:
 {'id': '/subscriptions/d40c7e06-9290-4055-8fa6-0ab11d40ff93/resourceGroups/kexin/providers/Microsoft.Network/networkSecurityG

## Update network interface with new IP configuration

In [9]:
# update to new ip
ip_config.public_ip_address = new_ip

# update network interface IP configration
update_op = network_client.network_interfaces.begin_create_or_update(
    resource_group_name=RESOURCE_GROUP,
    network_interface_name=ni_name,
    parameters={
        "location": LOCATION,
        # important: the nsg will be empty after update if not specified here
        "network_security_group": nsg.as_dict(),
        "ip_configurations": [ip_config.as_dict()]
    }
)
# wait for update operation to be completed
update_op.result()
print(f"Successfully updated to new IP: {new_ip_address}")

Successfully updated to new IP: 48.218.144.2


## Delete old IP by name

In [10]:
# delete old IP by name
delete_op = network_client.public_ip_addresses.begin_delete(RESOURCE_GROUP, old_ip_name)
# no need to wait for delete op to be completed
# delete_op.result()
print(f"Successfully deleted old IP: {old_ip_name}")

Successfully deleted old IP: 2024_0120_2216
